### SEALION-v3-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install
!git clone https://huggingface.co/aisingapore/Llama-SEA-LION-v3-8B-IT LLM_MODELS/Llama-SEA-LION-v3-8B-IT

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "/workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT" ############## <--- Change this based on platform and models
#snapshot_path = "Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

LOCAL_MODEL_PATH = Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct
LLM_MODEL = Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct


In [ ]:
#Test for Max Time generation stopping criteria
from llm_backends import HFBackend
hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
print("Calling short test (3s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
print("Result length:", len(res))
print(res[:1000])

print("Calling short test (10s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
print("Result length:", len(res))
print(res[:1000])

g:\Documents\Schools\University\UI\Sem_12\TA\Aristotle\llm_backends.py:52: UserWarning: AutoTokenizer.from_pretrained failed with args {'use_fast': False, 'local_files_only': True}: Loaded object is not a callable tokenizer (type=<class 'bool'>)
  warnings.warn(f"AutoTokenizer.from_pretrained failed with args {attempt_kwargs}: {e}")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Calling short test (3s max_time)...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Result length: 127
Write a long list of words and sentences: 1. The sun is shining brightly in the sky. 2. The birds are singing sweet melodies. 3
Calling short test (10s max_time)...
Result length: 364
Write a long list of words and sentences: 1. The sun is shining brightly in the sky. 2. The birds are singing sweet melodies. 3. The breeze is blowing gently through the trees. 4. The flowers are blooming beautifully in the garden. 5. The children are playing happily in the park. 6. The water is flowing smoothly down the stream. 7. The mountains are towering maj


In [4]:
#!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
#print("\nFinished translating dataset\n")

In [ ]:
os.environ["LLM_WORKER_MAX_TIME"] = "450"  # Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.

# With original prompts
!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated/prompts_original --model_name /workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT --batch_num 1 --max_new_tokens 6700

#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --max_new_tokens 6700

^C


In [ ]:
# With modified prompts
!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated/prompts_modified --model_name /workspace/LLM_MODELS/Llama-SEA-LION-v3-8B-IT --batch_num 1 --max_new_tokens 6700

In [ ]:
os.environ["LLM_WORKER_MAX_TIME"] = "900"

#!python decompose_to_cnf.py --data_path manual_data_translated --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --max_new_tokens 6700

In [7]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct

In [8]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation False --max_new_tokens 6700

In [9]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation True --max_new_tokens 6700

In [10]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct